In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Conv2D, MaxPooling2D, Activation, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import applications
from keras import optimizers
from keras import callbacks
import glob
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
from keras import backend as K

from skimage import color, transform, exposure
from skimage.io import imread
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# dimensions of our images.
img_width, img_height = 512, 512

train_data_dir = '../data/model/key19/train'
validation_data_dir = '../data/model/key19/validation'
test_data_dir = '../data/model/key19/test'
nb_train_samples = int(len(glob.glob(train_data_dir+'/*/*')))
nb_validation_samples = int(len(glob.glob(validation_data_dir+'/*/*')))
nb_test_samples = int(len(glob.glob(test_data_dir+'/*/*')))
epochs = 100
batch_size = 8
print('Found {} training samples, {} validation samples and {} test samples'.format(nb_train_samples,nb_validation_samples, nb_test_samples))

In [ ]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [ ]:
reg = 0.001

model = Sequential()

model.add(Conv2D(100, (7, 7), input_shape=input_shape, padding='same',
                kernel_regularizer=regularizers.l2(reg)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(150, (4, 4), padding='same',
                kernel_regularizer=regularizers.l2(reg)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(250, (4, 4), padding='same',
                kernel_regularizer=regularizers.l2(reg)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(6))
model.add(Activation('softmax'))

AdamOpt = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy',
              optimizer=AdamOpt,
              metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
def preprocessing(img):
    img /= 255.
    img_eq = exposure.equalize_adapthist(img, clip_limit=0.3)
    

    return img_eq

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocessing,
        rotation_range=15,
        width_shift_range=0.15,
        height_shift_range=0.15,
        shear_range=0.15,
        zoom_range=0.15,
        fill_mode='nearest')

validation_datagen = ImageDataGenerator(preprocessing_function=preprocessing)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
class PlotLosses(callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.train_acc = []
        self.val_acc = []
        
        #self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.train_acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        
        
        self.i += 1
        
        clear_output(wait=True)
        plt.figure()
        plt.plot(self.x, self.losses, label="train_loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show()
        plt.figure()
        plt.plot(self.x, self.train_acc, label="train_acc")
        plt.plot(self.x, self.val_acc, label="val_acc")
        plt.legend()
        plt.show();
        
plot_losses = PlotLosses()

In [ ]:
checkpointer = ModelCheckpoint(filepath='../saved_weights/Traffic_histeq_key19_weights.hdf5', verbose=1, save_best_only=True)

In [ ]:
train_history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[checkpointer, plot_losses])

In [ ]:
model.load_weights('../saved_weights/Traffic_key19_weights.hdf5')

In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=Adapthisteq)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    shuffle = False,
    class_mode='categorical')

In [ ]:
test_pred_prob = model.predict_generator(test_generator,steps = nb_test_samples, verbose=1)
test_pred_labels = np.argmax(test_pred_prob, axis=-1)
acc = 1-np.sum(test_pred_labels!=test_generator.classes[:nb_test_samples])/nb_test_samples
print(acc)

In [ ]:
idx = np.array(np.where(test_pred_labels!=test_generator.classes))
print('Failed {} images out of {} images'.format(idx.shape[1],nb_test_samples))
for i in range(0,idx.shape[1]):
    print('Classified as pos {}'.format(test_pred_labels[idx[0,i]]+1))
    filename = '../data/model/key19/test/'+test_generator.filenames[idx[0,i]]
    image = imread(filename)
    plt.imshow(image)
    plt.title(filename)
    plt.axis('off')
    plt.show()

In [ ]:
model.save('traffic.h5')